In [14]:
import numpy as np


class DataReader:
    def __init__(self, path, seq_length):
        self.fp = open(path, "r")        
        self.data = self.fp.read()
        chars = list(set(self.data))
        self.char_to_ix = {ch:i for (i,ch) in enumerate(chars)}
        self.ix_to_char = {i:ch for (i,ch) in enumerate(chars)}
        self.data_size = len(self.data)
        self.vocab_size = len(chars)
        self.pointer = 0
        self.seq_length = seq_length

    def next_batch(self):
        input_start = self.pointer
        input_end = self.pointer + self.seq_length
        inputs = [self.char_to_ix[ch] for ch in self.data[input_start:input_end]]
        targets = [self.char_to_ix[ch] for ch in self.data[input_start+1:input_end+1]]
        self.pointer += self.seq_length
        if self.pointer + self.seq_length + 1 >= self.data_size:
            self.pointer = 0
        return inputs, targets

    def just_started(self):
        return self.pointer == 0

    def close(self):
        self.fp.close()

In [15]:
class RNN:
    def __init__(self, hidden_size, vocab_size, seq_length, learning_rate):
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.seq_length = seq_length
        self.learning_rate = learning_rate
        self.U = np.random.uniform(-np.sqrt(1./vocab_size), np.sqrt(1./vocab_size), (hidden_size, vocab_size))
        self.V = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (vocab_size, hidden_size))
        self.W = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (hidden_size, hidden_size))
        self.b = np.zeros((hidden_size, 1))
        self.c = np.zeros((vocab_size, 1)) 
        self.mU = np.zeros_like(self.U)
        self.mW = np.zeros_like(self.W)
        self.mV = np.zeros_like(self.V)
        self.mb = np.zeros_like(self.b)
        self.mc = np.zeros_like(self.c)

    def softmax(self, x):
        p = np.exp(x- np.max(x))
        return p / np.sum(p)
        
    def forward(self, inputs, hprev):
            xs, hs, os, ycap = {}, {}, {}, {}
            hs[-1] = np.copy(hprev)
            for t in range(len(inputs)):
                xs[t] = np.zeros((self.vocab_size,1))
                xs[t][inputs[t]] = 1
                hs[t] = np.tanh(np.dot(self.U,xs[t]) + np.dot(self.W,hs[t-1]) + self.b)
                os[t] = np.dot(self.V,hs[t]) + self.c 
                ycap[t] = self.softmax(os[t])
            return xs, hs, ycap
        
    def backward(self, xs, hs, ps, targets):

            dU, dW, dV = np.zeros_like(self.U), np.zeros_like(self.W), np.zeros_like(self.V)
            db, dc = np.zeros_like(self.b), np.zeros_like(self.c)
            dhnext = np.zeros_like(hs[0])
            for t in reversed(range(self.seq_length)):       
                dy = np.copy(ps[t])
                dy[targets[t]] -= 1 
                dV += np.dot(dy, hs[t].T)
                dc +=dy # need to check
                dh = np.dot(self.V.T, dy) + dhnext 
                dhrec = (1 - hs[t] * hs[t]) * dh 
                db += dhrec
                dU += np.dot(dhrec, xs[t].T)
                dW += np.dot(dhrec, hs[t-1].T)
                dhnext = np.dot(self.W.T, dhrec)
            for dparam in [dU, dW, dV, db, dc]:
                np.clip(dparam, -5, 5, out=dparam) 
            return dU, dW, dV, db, dc
    
    def loss(self, ps, targets):
            return sum(-np.log(ps[t][targets[t],0]) for t in range(self.seq_length))      
    
    def update_model(self, dU, dW, dV, db, dc):
        for param, dparam, mem in zip([self.U, self.W, self.V, self.b, self.c],
                                  [dU, dW, dV, db, dc],
                                  [self.mU, self.mW, self.mV, self.mb, self.mc]):
            mem += dparam*dparam
            param += -self.learning_rate*dparam/np.sqrt(mem+1e-8)
                
                
    def sample(self, h, seed_ix, n):
            x = np.zeros((self.vocab_size, 1))
            x[seed_ix] = 1
            ixes = []
            for t in range(n):
                h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
                y = np.dot(self.V, h) + self.c
                p = np.exp(y)/np.sum(np.exp(y))
                ix = np.random.choice(range(self.vocab_size), p = p.ravel())
                x = np.zeros((self.vocab_size,1))
                x[ix] = 1
                ixes.append(ix)
            return ixes

    def train(self, data_reader):
            iter_num = 0
            threshold = 5
            smooth_loss = -np.log(1.0/data_reader.vocab_size)*self.seq_length
            while (smooth_loss > threshold):
                if data_reader.just_started():
                    hprev = np.zeros((self.hidden_size,1))
                inputs, targets = data_reader.next_batch()
                xs, hs, ps = self.forward(inputs, hprev)
                dU, dW, dV, db, dc = self.backward(xs, hs, ps, targets)
                loss = self.loss(ps, targets)
                self.update_model(dU, dW, dV, db, dc)
                smooth_loss = smooth_loss*0.999 + loss*0.001
                hprev = hs[self.seq_length-1]
                if not iter_num%500:
                    sample_ix = self.sample(hprev, inputs[0], 200)
                    print( ''.join(data_reader.ix_to_char[ix] for ix in sample_ix))
                    print( "\n\niter :%d, loss:%f"%(iter_num, smooth_loss))
                iter_num += 1

    def predict(self, data_reader, start, n):
        x = np.zeros((self.vocab_size, 1))
        chars = [ch for ch in start]
        ixes = []
        for i in range(len(chars)):
            ix = data_reader.char_to_ix[chars[i]]
            x[ix] = 1
            ixes.append(ix)
        h = np.zeros((self.hidden_size,1))
        for t in range(n):
            h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
            y = np.dot(self.V, h) + self.c
            p = np.exp(y)/np.sum(np.exp(y))
            ix = np.random.choice(range(self.vocab_size), p = p.ravel())
            x = np.zeros((self.vocab_size,1))
            x[ix] = 1
            ixes.append(ix)
        txt = ''.join(data_reader.ix_to_char[i] for i in ixes)
        return txt

In [23]:
data_reader = DataReader("input.txt", seq_length)

In [24]:
seq_length = 20
#read text from the "input.txt" file
data_reader = DataReader("input.txt", seq_length)
rnn = RNN(hidden_size=100, vocab_size=data_reader.vocab_size,seq_length=seq_length,learning_rate=1e-1)
rnn.train(data_reader)

eHuwr at abfn& nHl2:an0 DnDn  nwunrDJ: w;:TSn;e   !r&wf . 0 MbhSrsL tnpwgnk wosn9E   rJ s&f-wna 1 Daa,I dd c.b'S n2 M x:nNmaOg1o  ns antNEs upT;e  nn in InIwanna v sMl ra gay09C“na!y 
Dc2    n  i PoaB


iter :0, loss:81.550511
wo lot rorae tuIo  li1 see  ra e tc soul l r rep wnth beid tori !e e l Ho s t our, atoug fbn if avss wd oon  .e t 
  be oroerlhdr thee re
 tse wo e rofwiu loh s he  t nbaner mr foe âlul w ey wo seermd


iter :500, loss:74.459109
fe doomde  cloogar, wrtt weme no! tyre doâ her, thenhe hillehoendomer inrwe bimee sinoefes romee eaU nane f tas to iafra ;efer faeme nos ah who deoave cidonfouwe filvitopg woade0k2bhe, cyoadgdt“ Wuaw 


iter :1000, loss:66.116934
Bpbilvo Saboutinly thin alte marnga Cond the, ta thend tte,. Ad wot mipxolheor
T9 and tSd and anm th fot an, wouseiund ,asscand mad amee hasame, the tegresthey Ahetiley plin thimsh,in ptoâmer alfeeve 


iter :1500, loss:58.719228
 tople tor mand taghme th. in woub. peime tethed msy tata d bhiptibs.idy wa:n is he 

In [27]:
rnn.predict(data_reader, 'I', 500)

"In a town in they came un a nain unt. ridh of and mand themurto a tice mabe toth in by came ut to masther, wint stoustind hid in a his the bagntied got iutwer, and said:  Open Sesame!' Evered ts a donge wull this ifrieved Ali to a tresas paclin mothed Ali Baba chut one and dormich oputill and the do a homed. At siatreid shim ant in VertWeas wno munt to the door horseat ass. He whowind the stand ne fore tloresind the door Ali Baba hidh wiunted theid tofe ald carn. Thein wes ale shed toth uples.\n\nI"